# Imports

In [ ]:
import csv
import pandas as pd
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy.stats import cauchy, bernoulli, binom
import math



from glob import glob
import glob
import re
import os, fnmatch
import csv
from io import StringIO



In [3]:
import numpy as np
Theta = [1,0.7,0.5]
self.thetas_k3_ = np.ones(B, dtype=float)*Theta[2]

# Example NumPy array
my_np_array = np.array([3, 1,9, 4, 1, 5, 9, 2, 6, 5, 3, 5])

# Find the index of the maximum element
index_of_max_element = np.argmax(my_np_array)

print("Original NumPy array:", my_np_array)
print("Index of the maximum element:", index_of_max_element)


Original NumPy array: [3 1 9 4 1 5 9 2 6 5 3 5]
Index of the maximum element: 2


## Class containers:

In [ ]:
class container : 
    def __init__(self,k,V,Q_max):        
        self.k = k
        self.V = V
        self.Q_max = Q_max
        self.mu = 0
        self.w = 0
        self.Q = 0
        self.Q_ema = 0
        self.D = 0
        
    
#     def departure_offer(self):
#         self.D = int(self.mu/12000) #12000 = pkt_size_1500 bytes * (8 bits / 1 byte)
        
#         if (self.Q_ <= self.V) : 
#             self.Q = self.Q_
#         else : 
#             self.Q = self.Q_max
        
#     def weight(self):
#         self.w = math.log(1 + (self.Q/self.V))
        

RT_lxc = []

#RT_lxc.append(container(1,1,200,1000))

#print(RT_lxc[0].V)

R = 2

for i in range(R) : 
    RT_lxc.append(container(1,200,1000))
    
Q = []
for lxc in RT_lxc :
    print(lxc.Q,lxc.V)
    Q.append(lxc.Q)
RT_lxc[0].Q = 40
print(Q)

    


    
    

## Queuing dynamics

In [ ]:
def queuing_dynamics(lxc,lambda_) :    
    D = lxc.D; Q = lxc.Q; Q_max = lxc.Q_max
    
    # arrivals:
    a = np.random.binomial(1, lambda_)        
    
    # dequeue
    if (D > 0) : d = 1;
    else d = 0;
    
    # queuing dynamics
    if Q < Q_max :
        lxc.Q = max(Q - d,0) + a
    else :
        lxc.Q = max(Q - d,0)
        
    # departures update: the offered amount of packets that can be transmitted in slot t 
    # decreases at the sub-slot granularity (i.e., suppose 66 pps can be transmitted in one second. 
    #At time-slot t - d_t the number of packets that can be transmitted until the next second decreases 
    #at the subtime slot d_t rate)
    lxc.D = D - d  

    


In [3]:
from scipy.stats import poisson
import matplotlib.pyplot as plt
import numpy as np

# Set the arrival rate (lambda)
lambda_value = 5000000

# Create a Poisson distribution object
poisson_dist = poisson(mu=lambda_value)

# Generate random samples
random_samples = poisson_dist.rvs(1)
print(random_samples)

a = 3
print(a)
a = min(a, 2)

print(a)


[5000116]
3
2


In [3]:
class MyClass:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def display_info(self):
        print(f"Name: {self.name}, Age: {self.age}")

# Create an empty list to store objects
my_objects = []

# Use a loop to instantiate objects and add them to the list
for i in range(3):  # You can adjust the range based on how many objects you want
    name = 'sisas'+str(i)
    age = 10 + i
    obj = MyClass(name, age)
    my_objects.append(obj)

for obj in my_objects:
    obj.display_info()


Name: a, Age: 5
Name: a, Age: 8
Name: a, Age: 9


## INIT system

In [ ]:
def init_system(d_t,d_t_p,d_t_m,LXC_RT,LXC_PR,LXC_BE,BW,pkt_size,alpha)  :    
    D = (1/N)*BW*(1/pkt_size)
    
    for lxc in LXC_RT : 
        lxc.D = D;
        t_m = 0; t_p = 0;
        
        while(t_m < d_t) : 
            while (t_p < d_t_m) :
                queuing_dynamics(lxc,lambda_)
                t_p = t_p + d_t_p
            ema(lxc, alpha)
            t_p = 0;
            t_m = t_m + d_t_m;
            
    for lxc in LXC_PR : 
        lxc.D = D;
        t_m = 0; t_p = 0;
        
        while(t_m < d_t) : 
            while (t_p < d_t_m) :
                queuing_dynamics(lxc,lambda_)
                t_p = t_p + d_t_p
            ema(lxc, alpha)
            t_p = 0;
            t_m = t_m + d_t_m;
            
    for lxc in LXC_BE : 
        lxc.D = D;
        t_m = 0; t_p = 0;
        
        while(t_m < d_t) : 
            while (t_p < d_t_m) :
                queuing_dynamics(lxc,lambda_)
                t_p = t_p + d_t_p
            ema(lxc, alpha)
            t_p = 0;
            t_m = t_m + d_t_m;



# NBA algorithm

## Backlog monitoring

In [ ]:
def ema_backlog(lxc, alpha) :
    Q = lxc.Q; ema = lxc.Q_ema    
    lxc.Q_ema = (alpha*Q) + ((1-alpha)*ema)

## Weight computation

In [ ]:
def nba_weight(lxc) :
    Q = lxc.Q_ema; V= lxc.V
    lxc.w = np.log(1 + Q/V)


## Bandwidth allocation

In [ ]:
def nba_bandwidth_allocation(LXC_RT,LXC_PR,LXC_BE,BW,packet_size) :
    sum(obj.value for obj in my_array)
    sum_rt_w = sum(lxc.w for lxc in LXC_RT)
    sum_pr_w = sum(lxc.w for lxc in LXC_PR)
    sum_be_w = sum(lxc.w for lxc in LXC_BE)
    
    sum_w = sum_rt_w + sum_pr_w + sum_be_w
    
    for lxc in LXC_RT : 
        w_n = lxc.w        
        lxc.mu = (w_n/sum_w)*C
        lxc.D = int(lxc.mu/(packet_size*8))
    
    for lxc in LXC_PR : 
        w_n = lxc.w        
        lxc.mu = (w_n/sum_w)*C
        lxc.D = int(lxc.mu/(packet_size*8))
        
    for lxc in LXC_BE : 
        w_n = lxc.w        
        lxc.mu = (w_n/sum_w)*C
        lxc.D = int(lxc.mu/(packet_size*8))
        
    
    

# Main program

In [ ]:
# t = 0; d_t = 1;          # system time_slot
# t_1 = 0; d_t_1 = 0.1;    # sampling monitoring 
# t_2 = 0; d_t_2 = 0.01;   # packet time_slot
# T_max = 100;

t = 0; d_t = 1;          # system time_slot
t_m = 0; d_t_m = 0.1;    # sampling monitoring 
t_p = 0; d_t_p = 0.01;   # packet time_slot
T_max = 5;


p = 0.65
D = 66

lambda_ = 0.65

RT = len(LXC_RT)
PR = len(LXC_PR)
BE = len(LXC_BE)
N = RT + PR + BE
pkt_size = 8*1500 #(pkt size in bytes is 1500)

#init system
init_system(d_t,t_p,d_t_p,t_m,d_t_m,LXC_RT,LXC_PR,LXC_BE,BW)  



while (t < T_max) :        
    
    while(t_m < d_t) : 
        # We use this loop to mark the time at which monitoring take place. This way we avoid the conditional comparison 
        # whether the packet-time-slot has reached the monitoring period. The nested loop below is the actual time-slot
        # evolution at packet granularity
        
        while (t_p < d_t_m) :
            
            ##-> Computing queuing dynamics for each container
            for lxc in LXC_RT :             
                queuing_dynamics(lxc,lam)
            
            for lxc in LXC_PR :             
                queuing_dynamics(lxc,lam)
                
            for lxc in LXC_BE :             
                queuing_dynamics(lxc,lam)            
            t_p = t_p + d_t_p
        
        ##-> Backlog monitoring: every monitoring time-slot t_m (where d_t_m < d_t_m < d_t) 
        # we compute the backlog of each container as the EMA of the recent backlog and old measurements.        
        for lxc in LXC_RT :
            ema_backlog(lxc,alpha)
            
        for lxc in LXC_PR :
            ema_backlog(lxc,alpha)
                
        for lxc in LXC_BE :
            ema_backlog(lxc,alpha)                    
        
        # packet sub time slot reset       
        t_p = 0;
        t_m = t_m + d_t_m;
    
    ##-> NBA algorithm is computed every system time-slot d_t
    for lxc in LXC_RT :
        nba_weight(lxc)
            
    for lxc in LXC_PR :
        nba_weight(lxc)
                
    for lxc in LXC_BE :
        nba_weight(lxc)
    
    
    nba_bandwidth_allocation(LXC_RT,LXC_PR,LXC_BE)
    t_m = 0
    print("NBA")
    D = 66
    t = t + d_t
    
    
    
    
    

In [ ]:
Q_11 = np.array((10,20,30,40,50,60,80,100), dtype=float)
Q_21 = np.array((10,20,30,40,50,60,80,100), dtype=float)
Q_31 = np.array((10,20,30,40,50,60,80,100), dtype=float)
decay = np.array((0.14,0.28,0.56,0.4,0.5,0.6,0.7,0.8,0.9,1))


In [ ]:
fig, ax = plt.subplots()

# Plot Q_11^decay for each decay value
for d in decay:
    Q_11_decay = Q_11 ** d
    
    ax.plot(Q_11, Q_11_decay, label=f'decay = {d}')

# Add labels and a legend
ax.set_xlabel('Q_11')
ax.set_ylabel('Q_11_decay')
ax.legend()

# Show the plot
plt.show()


In [ ]:
Q_11 = np.array((10,20,30,40,50,60,80,100), dtype=float)
print('Q_11',Q_11)
print('Q_11^0.01',Q_11 ** (1+(1/250)))
print('log_Q_11',np.log(1+Q_11/250))
#print('log_Q_11^0.01',np.log(Q_11 ** 0.01))
print('log_1+Q_11',np.log(1+Q_11))
print('--------')
Q_21 = np.array((10,20,30,40,50,60,80,100), dtype=float)
print('Q_21',Q_21)
print('Q_21^0.01',Q_21 ** (1+(1/500)))
print('log_Q_21',np.log(1+Q_21/500))
print('1+log_Q_21',np.log(1+Q_21))

#print('log_Q_21^0.01',np.log(Q_21 ** 0.01))
print('--------')

#Q_31 = np.array((20,200,400,600,700,800,1000), dtype=float)
Q_31 = np.array((10,20,30,40,50,60,80,100), dtype=float)
print('Q_31',Q_31)
print('Q_31^0.01',Q_31 ** 0.1)
print('log_Q_31',np.log(1+Q_31/1000))
#print('log_Q_31^0.01',np.log(Q_31 ** 0.1))
print('1+log_Q_31',np.log(1+Q_31))
print('--------')


In [ ]:
#Q_11 = np.array((10,20,30,40,50,60,70,80,100), dtype=float)
Q_11 = np.array((10,20,30,40,10,30,20,10,10), dtype=float)
V_1 = 250; w_1 = 1

Q_21 = np.array((10,20,30,40,50,60,70,80,100), dtype=float)
#Q_21 = np.array((10,20,30,50,100,200,300,400,500), dtype=float)
V_2 = 2*V_1; w_2 = V_1/V_2


Q_31 = np.array((10,20,30,40,50,60,70,80,100), dtype=float)
#Q_31 = np.array((10,20,100,200,400,600,700,800,1000), dtype=float)
V_3 = 3*V_1; w_3 = V_1/V_3



'''
Q_11 = np.array((100,200,300,400,500,600,800,1000), dtype=float)
Q_21 = np.array((100,200,300,400,500,600,800,1000), dtype=float)
Q_31 = np.array((100,200,300,400,500,600,800,1000), dtype=float)
'''

fig, ax = plt.subplots()

# Calculate the logarithm of the arrays
#W_11 = w_1*Q_11
#W_11 = Q_11 ** w_1
#W_11 = Q_11 ** 0.01
#W_11 = Q_11 ** 1/250
#W_11 = Q_11 ** (1+1/250)
#W_11 = Q_11 ** (-1/250)
#W_11 = np.log(1+ (Q_11 ** 1/250)
#W_11 = np.log(Q_11 ** (1+(1/250)))
#W_11 = np.log(Q_11 ** ((1/250)))
W_11 = np.log(1 + (Q_11/V_1))
#W_11 = np.log(1 + (np.square(Q_11)/(V_1*V_1)))
#W_11 = Q_11/np.log(1_3)
#W_11 = w_1*np.log(1 + (Q_11))
#W_11 = np.log(Q_11/250)
#W_11 = np.log((1/V_1) + Q_11)
#W_11 = (Q_11/V_1)
#W_11 = Q_11/(250*250)

print('Q_11',Q_11)
print('W_11',W_11)


#W_21 = w_2*Q_21
#W_21 = Q_21 ** w_2
#W_21 = np.log(Q_21 ** 0.05)
#W_21 = Q_21 ** 1/500
#W_21 = Q_21 ** (1+1/500)
#W_21 = Q_21 ** (-1/500)
#W_21 = np.log(1+ (Q_21 ** 1/500))
#W_21 = np.log(Q_21 ** (1+(1/500)))
#W_21 = np.log(Q_21 ** ((1/500)))
W_21 = np.log(1 + (Q_21/V_2))
#W_21 = np.log(1 + (np.square(Q_21)/(V_2*V_2)))
#W_21 = Q_21/np.log(V_2)
#W_21 = w_2*np.log(1 + (Q_21/V_2))
#W_21 = w_2*np.log(1 + (Q_21))
#W_21 = np.log(Q_21/500)
#W_21 = np.log((1/V_2) + Q_21)
#W_21 = Q_21/V_2

print('Q_21',Q_21)
print('W_21',W_21)


#W_31 = w_3*Q_31
#W_31 = Q_31 ** w_3
#W_31 = np.log(Q_31 ** 0.1)
#W_31 = Q_31 ** 1/1000
#W_31 = Q_31 ** (1+1/1000)
#W_31 = Q_31 ** (-1/1000)
#W_31 = np.log(1+ (Q_31 ** 1/1000))
#W_31 = np.log(Q_31 ** (1+(1/1000)))
#W_31 = np.log(Q_31 ** ((1/1000)))
W_31 = np.log(1 + (Q_31/V_3))
#W_31 = np.log(1 + (np.square(Q_31)/(V_3*V_3)))
#W_31 = Q_31/np.log(V_3)
#W_31 = w_3*np.log(1 + (Q_31/V_3))
#W_31 = w_3*np.log(1 + (Q_31))
#W_31 = np.log(1/(V_3) + Q_31)
#W_31 = np.log(Q_31/750)
#W_31 = Q_31/500
#W_31 = Q_31/V_3

print('Q_31',Q_31)
print('W_31',W_31)


x = np.array((1,2,3,4,5,6,7,8,9), dtype=float)

BW_11 = W_11/(W_11+W_21+ W_31)
print('BW_11',BW_11)

BW_21 = W_21/(W_11+W_21+ W_31)
print('BW_21',BW_21)

BW_31 = W_31/(W_11+W_21+ W_31)
print('BW_31',BW_31)


# Plotting
ax.plot(x,BW_11, label='W_11(10,20,30,40,50,60,80,100)')
ax.plot(x,BW_21, label='W_21(10,20,50,100,200,300,400,500)')
ax.plot(x,BW_31, label='W_31(10,20,100,200,400,600,800,1000)')
ax.set_xlabel('Q_kn')
ax.set_ylabel('BW')
#ax.set_xlim(0,1000)
plt.legend()
plt.show()


In [ ]:

my_array = np.array([1, 2, 3, 4, 5])

squared_array = np.square(my_array)

print(squared_array)


In [ ]:
plt.plot(Q_11, Q_11**alpha[0], label='Q_11^decay[0]')
plt.plot(Q_21, Q_21**alpha[1], label='Q_21^decay[1]')
plt.plot(Q_31, Q_31**alpha[2], label='Q_31^decay[2]')

plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
Q_11 = np.array((10,20,30,40,50,60,80,100), dtype=float)
k_1 = -250
Q_11_ = np.power(Q_11, k_1)
Q_21 = np.array((10,20,30,40,50,60,80,100), dtype=float)
k_2 = -500
Q_21_ = np.power(Q_21, k_2)
Q_31 = np.array((10,20,30,40,50,60,80,100), dtype=float)
k_3 = -750
Q_31_ = np.power(Q_31, k_3)

#decay = np.array((0.01,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))

decay = np.array((0.01,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1))


In [ ]:
fig, ax = plt.subplots()

   
ax.plot(Q_11, Q_11_, label=f'k_1 = {k_1}')
ax.plot(Q_21, Q_21_, label=f'k_2 = {k_2}')

# Add labels and a legend
ax.set_xlabel('Q_11')
ax.set_ylabel('Q_11_')
ax.legend()

# Show the plot
plt.show()


# Columns

In [ ]:
rt_column_names = []
for i in range(1, P+1, 1):
    rt_column_names.append("rt_lxc_"+str(i))
print(rt_column_names)



pr_column_names = []
for i in range(1, P+1, 1):
    pr_column_names.append("pr_lxc_"+str(i))
print(pr_column_names)


be_column_names = []
for i in range(1, B+1, 1):
    be_column_names.append("be_lxc_"+str(i))
print(be_column_names)


# Bandwidht Allocation



## RT LXC

In [ ]:
rt_mu_02_df = pd.read_csv(path + "mu_rt_2.csv",names = rt_column_names) 
rt_mu_02_df["avg"] = rt_mu_02_df.mean(axis=1)
#print(rt_mu_02_df)


rt_mu_04_df = pd.read_csv(path + "mu_rt_4.csv",names = rt_column_names) 
rt_mu_04_df["avg"] = rt_mu_04_df.mean(axis=1)
#print(rt_mu_04_df)

rt_mu_06_df = pd.read_csv(path + "mu_rt_6.csv",names = rt_column_names) 
rt_mu_06_df["avg"] = rt_mu_06_df.mean(axis=1)
#print(rt_mu_06_df)


rt_mu_08_df = pd.read_csv(path + "mu_rt_8.csv",names = rt_column_names) 
rt_mu_08_df["avg"] = rt_mu_08_df.mean(axis=1)
#print(rt_mu_08_df)

rt_mu_10_df = pd.read_csv(path + "mu_rt_10.csv",names = rt_column_names) 
rt_mu_10_df["avg"] = rt_mu_10_df.mean(axis=1)
#print(rt_mu_10_df)


rt_mu_mean_ = np.array([rt_mu_02_df["avg"].mean(axis=0),\
                       rt_mu_04_df["avg"].mean(axis=0),\
                       rt_mu_06_df["avg"].mean(axis=0),\
                       rt_mu_08_df["avg"].mean(axis=0),\
                       rt_mu_10_df["avg"].mean(axis=0)])
#rt_mu_mean_df = pd.DataFrame(rt_mu_mean_, index=lambda_,columns = ["LXC-TS"])
rt_mu_mean_df = pd.DataFrame(rt_mu_mean_, columns = ["LXC-TS"])
rt_mu_mean_df.head()



## PR LXC


In [ ]:
pr_mu_02_df = pd.read_csv(path + "mu_pr_2.csv",names = pr_column_names) 
pr_mu_02_df["avg"] = pr_mu_02_df.mean(axis=1)
#print(pr_mu_02_df)


pr_mu_04_df = pd.read_csv(path + "mu_pr_4.csv",names = pr_column_names) 
pr_mu_04_df["avg"] = pr_mu_04_df.mean(axis=1)
#print(pr_mu_04_df)

pr_mu_06_df = pd.read_csv(path + "mu_pr_6.csv",names = pr_column_names) 
pr_mu_06_df["avg"] = pr_mu_06_df.mean(axis=1)
#print(pr_mu_06_df)


pr_mu_08_df = pd.read_csv(path + "mu_pr_8.csv",names = pr_column_names) 
pr_mu_08_df["avg"] = pr_mu_08_df.mean(axis=1)
#print(pr_mu_08_df)

pr_mu_10_df = pd.read_csv(path + "mu_pr_10.csv",names = pr_column_names) 
pr_mu_10_df["avg"] = pr_mu_10_df.mean(axis=1)
#print(pr_mu_10_df)


pr_mu_mean_ = np.array([pr_mu_02_df["avg"].mean(axis=0),\
                       pr_mu_04_df["avg"].mean(axis=0),\
                       pr_mu_06_df["avg"].mean(axis=0),\
                       pr_mu_08_df["avg"].mean(axis=0),\
                       pr_mu_10_df["avg"].mean(axis=0)])
#pr_mu_mean_df = pd.DataFrame(pr_mu_mean_, index=lambda_,columns = ["LXC-TS"])
pr_mu_mean_df = pd.DataFrame(pr_mu_mean_, columns = ["LXC-PR"])
pr_mu_mean_df.head()



# BE LXC

In [ ]:
be_mu_02_df = pd.read_csv(path + "mu_be_2.csv",names = be_column_names) 
be_mu_02_df["avg"] = be_mu_02_df.mean(axis=1)
#print(be_mu_02_df)


be_mu_04_df = pd.read_csv(path + "mu_be_4.csv",names = be_column_names) 
be_mu_04_df["avg"] = be_mu_04_df.mean(axis=1)
#print(be_mu_04_df)

be_mu_06_df = pd.read_csv(path + "mu_be_6.csv",names = be_column_names) 
be_mu_06_df["avg"] = be_mu_06_df.mean(axis=1)
#print(be_mu_06_df)


be_mu_08_df = pd.read_csv(path + "mu_be_8.csv",names = be_column_names) 
be_mu_08_df["avg"] = be_mu_08_df.mean(axis=1)
#print(be_mu_08_df)

be_mu_10_df = pd.read_csv(path + "mu_be_10.csv",names = be_column_names) 
be_mu_10_df["avg"] = be_mu_10_df.mean(axis=1)
#print(be_mu_10_df)


be_mu_mean_ = np.array([be_mu_02_df["avg"].mean(axis=0),\
                       be_mu_04_df["avg"].mean(axis=0),\
                       be_mu_06_df["avg"].mean(axis=0),\
                       be_mu_08_df["avg"].mean(axis=0),\
                       be_mu_10_df["avg"].mean(axis=0)])
#be_mu_mean_df = pd.DataFrame(be_mu_mean_, index=lambda_,columns = ["LXC-TS"])
be_mu_mean_df = pd.DataFrame(be_mu_mean_, columns = ["LXC-BE"])
be_mu_mean_df.head()



# Plots

In [ ]:
mu_mean = pd.concat([rt_mu_mean_df,pr_mu_mean_df,be_mu_mean_df],axis=1)
mu_mean['rate'] = lambda_
mu_mean = mu_mean.melt('rate', var_name='LXC', value_name='Mean Throughput')
print(mu_mean)

fig, ax = plt.subplots(1, 1, figsize=(4, 5))
#fig, ax = plt.subplots(2, 2, figsize=(10, 15),  gridspec_kw={'hspace': 0.2,'wspace': 0.2})
sns.set()
sns.despine(left=True)
sns.set_style("whitegrid")
sns.catplot(x="rate", y = "Mean Throughput", hue='LXC', data = mu_mean, kind='point',palette="Accent",ax=ax)
ax.tick_params(axis='x',labelsize=12)
ax.set_ylabel("Throughput [Mbps]",fontsize=14,fontweight='bold')

plt.setp(ax.get_xticklabels(), fontweight="bold")
plt.setp(ax.get_yticklabels(), fontweight="bold")
plt.savefig('no_penalty_1_1_5.png', format='png', dpi=1200)





'''
mu['x-axis'] = lambda_
mum = mu.melt('x-axis', var_name='cols', value_name='vals')


fig, ax = plt.subplots(1, 1, figsize=(4, 5))
#fig, ax = plt.subplots(2, 2, figsize=(10, 15),  gridspec_kw={'hspace': 0.2,'wspace': 0.2})
sns.set()
sns.despine(left=True)
sns.set_style("whitegrid")

sns.catplot(x="x-axis", y="vals", hue='cols', data=mum, kind='point',palette="Accent",ax=ax)

ax.tick_params(axis='x',labelsize=12)
ax.set_ylabel("Throughput [Mbps]",fontsize=14,fontweight='bold')

plt.setp(ax.get_xticklabels(), fontweight="bold")
plt.setp(ax.get_yticklabels(), fontweight="bold")
plt.savefig('sisas.png', format='png', dpi=1200)
'''

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 5))
#fig, ax = plt.subplots(2, 2, figsize=(10, 15),  gridspec_kw={'hspace': 0.2,'wspace': 0.2})
sns.set()
sns.despine(left=True)
sns.set_style("whitegrid")

vbbu2_e2e = pd.concat([vbbu2_cpu_sharing_df,\
                       vbbu2_cpu_affinity_df\
                      ],axis=0)
print(vbbu1_e2e)

sns.boxplot(data=vbbu2_e2e, palette="Accent",ax=ax,showfliers = False)
#ax.set_title('vBBU2\'s end to end Throughput', fontsize=12,fontweight='bold')
#ax.set_xlabel("")
ax.tick_params(axis='x',labelsize=12)
ax.set_ylabel("UE Throughput [Mbps]",fontsize=14,fontweight='bold')
#ax.set_yticklabels(ax.get_yticks(), weight="bold",fontsize = 12)
ax.set_ylim(0,14)
plt.setp(ax.get_xticklabels(), fontweight="bold")
plt.setp(ax.get_yticklabels(), fontweight="bold")

plt.savefig('vBBU2.svg', format='svg', dpi=1200)

